In [ ]:
pip install mne numpy matplotlib scikit-learn torch torchvision


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 96.3 MB/s eta 0:00:00


In [ ]:
import mne

# Choose subjects and recordings
subjects = [1, 2]  # example, you can add more subjects
recordings = [1]

# Fetch data
data_files = []
for subj in subjects:
    data_files.extend(mne.datasets.sleep_physionet.age.fetch_data(subjects=[subj], recording=recordings))

# Separate PSG and hypnogram files
psg_files = [f[0] for f in data_files]
hyp_files = [f[1] for f in data_files]

print("PSG files:", psg_files)
print("Hypnogram files:", hyp_files)


Using default location ~/mne_data for PHYSIONET_SLEEP...
Creating /root/mne_data


100%|█████████████████████████████████████| 51.1M/51.1M [00:00<00:00, 47.0GB/s]
100%|█████████████████████████████████████| 3.90k/3.90k [00:00<00:00, 2.74MB/s]

Download complete in 02m24s (48.7 MB)


Using default location ~/mne_data for PHYSIONET_SLEEP...


100%|█████████████████████████████████████| 51.1M/51.1M [00:00<00:00, 55.4GB/s]
100%|█████████████████████████████████████| 4.80k/4.80k [00:00<00:00, 4.07MB/s]


Download complete in 02m26s (48.8 MB)
PSG files: ['/root/mne_data/physionet-sleep-data/SC4011E0-PSG.edf', '/root/mne_data/physionet-sleep-data/SC4021E0-PSG.edf']
Hypnogram files: ['/root/mne_data/physionet-sleep-data/SC4011EH-Hypnogram.edf', '/root/mne_data/physionet-sleep-data/SC4021EH-Hypnogram.edf']


In [ ]:
import numpy as np
import mne

stage_mapping = {
    'Sleep stage W': 0,   # Wake
    'Sleep stage 1': 1,   # N1
    'Sleep stage 2': 2,   # N2
    'Sleep stage 3': 3,   # N3
    'Sleep stage 4': 3,   # merge stage 3 & 4 as N3
    'Sleep stage R': 4,   # REM
    # 'Sleep stage ?': -1   # Exclude unknown stage from mapping
}

def load_subject_data(psg_file, hyp_file, epoch_duration=30, channels=['EEG Fpz-Cz','EEG Pz-Oz']):
    raw = mne.io.read_raw_edf(psg_file, preload=True, verbose=False)

    # Pick only EEG channels for simplicity
    raw.pick_channels(channels)

    annotations = mne.read_annotations(hyp_file)
    raw.set_annotations(annotations)

    # Create events from annotations using the modified stage_mapping
    # Events corresponding to 'Sleep stage ?' will not be created
    events, event_id = mne.events_from_annotations(raw, event_id=stage_mapping)

    # With 'Sleep stage ?' excluded from mapping, all created events are valid
    valid_events = events

    # Epoching based on valid events
    # Use the event_id dictionary returned by events_from_annotations
    epochs = mne.Epochs(raw, valid_events, event_id=event_id, tmin=0., tmax=epoch_duration, baseline=None, preload=True, verbose=False)

    print("Type of epochs.events:", type(epochs.events))
    print("Shape of epochs.events:", epochs.events.shape)

    # Extract labels for each epoch using a list comprehension
    labels = np.array([event[2] for event in epochs.events])


    print("Epochs shape:", epochs.get_data().shape)   # (n_epochs, n_channels, samples_per_epoch)
    print("Labels shape:", labels.shape)

    return epochs.get_data(), labels

# Example: load first subject
X, y = load_subject_data(psg_files[0], hyp_files[0])
print("Epochs shape (after loading):", X.shape)
print("Labels shape (after loading):", y.shape)

/tmp/ipython-input-2843351411.py:15: RuntimeWarning: Channels contain different highpass filters. Highest filter setting will be stored.
  raw = mne.io.read_raw_edf(psg_file, preload=True, verbose=False)
/tmp/ipython-input-2843351411.py:15: RuntimeWarning: Channels contain different lowpass filters. Lowest filter setting will be stored.
  raw = mne.io.read_raw_edf(psg_file, preload=True, verbose=False)
/tmp/ipython-input-2843351411.py:15: RuntimeWarning: Highpass cutoff frequency 16.0 is greater than lowpass cutoff frequency 0.7, setting values to 0 and Nyquist.
  raw = mne.io.read_raw_edf(psg_file, preload=True, verbose=False)


NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Used Annotations descriptions: [np.str_('Sleep stage 1'), np.str_('Sleep stage 2'), np.str_('Sleep stage 3'), np.str_('Sleep stage 4'), np.str_('Sleep stage R'), np.str_('Sleep stage W')]
Type of epochs.events: <class 'numpy.ndarray'>
Shape of epochs.events: (125, 3)
Epochs shape: (125, 2, 3001)
Labels shape: (125,)
Epochs shape (after loading): (125, 2, 3001)
Labels shape (after loading): (125,)


/tmp/ipython-input-2843351411.py:21: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw.set_annotations(annotations)


In [ ]:
from sklearn.model_selection import train_test_split

# Flatten channels for simple classifier (or keep 2D for CNN)
X_flat = X.reshape(X.shape[0], -1)  # shape: (n_epochs, n_channels * samples_per_epoch)

# Truncate labels to match the number of epochs
y_aligned = y[:X_flat.shape[0]]

X_train, X_test, y_train, y_test = train_test_split(X_flat, y_aligned, test_size=0.2, random_state=42, stratify=y_aligned)
print("Train shape:", X_train.shape, y_train.shape)
print("Test shape:", X_test.shape, y_test.shape)

Train shape: (100, 6002) (100,)
Test shape: (25, 6002) (25,)


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Predictions
y_pred = clf.predict(X_test)

# Evaluation
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=['Wake','N1','N2','N3','REM']))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

Classification Report:
              precision    recall  f1-score   support

        Wake       0.00      0.00      0.00         3
          N1       0.50      0.33      0.40         6
          N2       0.35      0.75      0.48         8
          N3       0.50      0.33      0.40         6
         REM       0.00      0.00      0.00         2

    accuracy                           0.40        25
   macro avg       0.27      0.28      0.26        25
weighted avg       0.35      0.40      0.35        25

Confusion Matrix:
[[0 0 2 1 0]
 [0 2 4 0 0]
 [0 1 6 1 0]
 [0 0 4 2 0]
 [0 1 1 0 0]]


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
